<hr>
<h2> 0. Global Constance & Variable</h2>
Config global values
<hr>

In [1]:
_DATASET_ADDRESS_ = './dataset/'
_PICKLE_ADDRESS_ = './pickle/'

_DELTA_ = 0.00005

In [2]:
#기본 라이브러리
import pandas as pd
import numpy as np
import math
import time
import pickle
from tqdm.notebook import tqdm

tqdm.pandas()

# SVD 행렬축소
from sklearn.decomposition import TruncatedSVD

# 제작 함수
import func

In [3]:

#데이터 불러오기
ratings = pd.read_csv("u.data.txt", header=None, sep='\t')
ratings.columns = ['userid','movieid','rating','timestamp']
ratings = ratings.drop(['timestamp'], axis=1)
print("총 주어진 ratings 데이터 : ", ratings.shape)
rating_table= pd.pivot_table(ratings, values='rating', index=['userid'], columns=['movieid'])

총 주어진 ratings 데이터 :  (100000, 3)


In [4]:
# 80000 row
user_train_raw = pd.read_csv(_DATASET_ADDRESS_ + 'u1.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train_raw = user_train_raw[['userID','itemID','rating']]
user_train = user_train_raw.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)
user_train_raw = user_train_raw.pivot_table('rating', index = 'userID',columns = 'itemID')

# 20000 row
user_test = pd.read_csv(_DATASET_ADDRESS_ + 'u1.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]
user_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

<h3>MF define</h3>

In [5]:
import numpy

def matrix_factorization(R, P, Q, K, steps=300, alpha=0.0002, beta=0.02):

    

    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter
    '''
    Q = Q.T

    start = time.time()
    for step in range(steps):
        print('epoch: %d, time: %f'%(step, time.time()-start))
        for i in range(len(R)):
            for j in range(len(R[0])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[0])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [6]:
# N: num of User
try:
    with open(_PICKLE_ADDRESS_ + 'user_train.mf.pkl', 'rb') as f: 
        user_train_mf = pickle.load(f)
        print("Found existing Result...")
except FileNotFoundError:
    print("No existing Result, Calculating...")

    user_train_np = user_train.to_numpy()

    N = len(user_train_np)
    # M: num of Movie
    M = len(user_train_np[0])

    K = 5  #이거를 [5, 10, 15, 20, 25, 30]
    
    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)

    #   nP, nQ = matrix_factorization(user_train_np, P, Q, K)
    #user_train_mf = pd.DataFrame(nP@nQ.T)
    with open(_PICKLE_ADDRESS_ + 'user_train.mf.pkl', 'wb') as f:
        pickle.dump(user_train, f)

Found existing Result...


<h3>Train data :</h3>
<ol>
    <li>user_train_raw : rating with <strong>NaN</strong></li>
    <li>user_train     : rating filled with <strong>0</strong></li>
    <li>user_train_mf  : rating filled by <strong>MF</strong></li>
</ol>

In [7]:
#Test for loaded datas

user_train_mf.shape, user_train_raw.shape, user_train.shape

((943, 1650), (943, 1650), (943, 1650))

<br><br><br><br><br>
<br><br><br><br><br>
<hr>
<h2> 2. Group clustering with K-means</h2>
<hr>

In [8]:
from func import clustering

train_data, test_data, km_center = clustering(6, user_train_mf, user_train, user_test)

c:\_CSE\Python\Anaconda\envs\KDD\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [9]:
train_data.shape

(942, 1651)

<br><br><br><br><br>
<br><br><br><br><br>
<hr>
<h2> 3. Group recommendation</h2>
<ol>
    <li> 1) AVG</li>
    <li> 2) LM</li>
    <li> 3) Nash</li>
</ol>
<hr>

<h3>1) AVG</h3>

In [34]:
user_train_raw.shape

(943, 1650)

<h3>3) Nash</h3>

In [25]:
nash_train = user_train_raw
stat = []
for idx, item in nash_train.T.iterrows() :
    stat.append([item.count(), item.sum()])

In [33]:
stat[1609][0]

2

In [58]:
delta_train = []
for uidx, user in nash_train.iterrows() :
    iidx = 0
    tmp = []
    for item in user :
        userMean = stat[iidx][1] / stat[iidx][0]
        hasRated = not math.isnan(item)
        deltaSum = stat[iidx][1] - (item if hasRated else 0)
        deltaCount = stat[iidx][0] - (1 if hasRated else 0)
        tmp.append((userMean + _DELTA_) * deltaCount / (deltaSum + _DELTA_))
        iidx += 1
    delta_train.append(tmp)

In [65]:
delta_train = np.array(delta_train)
delta_train

array([[0.        , 0.        , 1.00029965, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.00095277, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.00095566, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])